# 概要
- このノートブックでは、ValueNetworkを学習させる。
- ネットワークの構造は以下のとおり。
    - 入力層：9チャネル
        - 黒石の位置(1)
        - 白石の位置(1)
        - 空白の位置(1)
        - 合法手の位置(1)
        - そこに打った場合、何個石を返せるか(1)
        - 隅の危険領域4マス×4隅をすべて1で埋める(1)
        - すべて1で埋める(1)
        - すべて0で埋める(1)
        - **手番情報：黒番ならすべて0で埋め、白番ならすべて1で埋める**(1)
    - 第1層：5x5のn_filters種類のフィルターとReLU関数
    - 第2-11層：3x3のn_filters種類のフィルターとReLU関数
    - 第12層：3x3のn_filters種類のフィルター
    - 第13層：1x1のn_filters種類のフィルター
    - 第14層：出力256個の全結合ネットワークとReLU関数
    - 第15層：出力1個の全結合ネットワークとtanh関数
- 学習データの作成方法は以下のとおり。（cf.AlphaGo解体新書p.171）
    - 1以上60以下の整数からランダムに数字を選択し、これをUとする。
    - ~~SL-PolicyNetworkをU-1回使って~~ランダムに手を選んで、U-1手目まで局面を進める。
    - 次のU手目は合法手の中からランダムに選択し局面を進め、この局面をSとする。
    - 局面Sからは、~~RL~~SLポリシーネットワークを使って、終局まで手を進める。最終的な勝敗をzとする。
    - 組(S,z)を学習データとする。

In [1]:
%%capture
!pip install creversi

In [2]:
from creversi import *

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from copy import copy
import gc

import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def board_to_array(board):
    """
    boardオブジェクトからndarrayに変換する関数(PolicyNetwork用)。
    第1チャンネルは黒石の位置、第2チャンネルに白石の位置、第3チャンネルに空白の位置、
    第4チャンネルに合法手の位置、第5チャンネルに返せる石の個数、第6チャンネルに隅=1、
    第7チャンネルに1埋め、第8チャンネルに0埋め。
    """
    b = np.zeros((8,8,8), dtype=np.float32)
    board.piece_planes(b)
    if not board.turn:
        b = b[[1,0,2,3,4,5,6,7],:,:]
    b[2] = np.where(b[0]+b[1]==1, 0, 1)
    legal_moves = list(board.legal_moves)
    if legal_moves != [64]:
        n_returns = []
        for move in legal_moves:
            board_ = copy(board)
            n_before = board_.opponent_piece_num()
            board_.move(move)
            n_after = board_.piece_num()
            n_returns.append(n_before-n_after)
        tmp = np.zeros(64)
        tmp[legal_moves] = n_returns
        tmp = tmp.reshape(8,8)
        b[3] = np.where(tmp > 0,1,0)
        b[4] = tmp
    b[5] = np.array([1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 
                     0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 
                     0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                     1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1.]).reshape(8,8)
    b[6] = 1
    return b

In [4]:
def board_to_array2(board):
    """
    boardオブジェクトからndarrayに変換する関数(ValueNetwork用)。
    第1チャネルは黒石の位置、第2チャネルに白石の位置、第3チャネルに空白の位置、
    第4チャネルに合法手の位置、第5チャネルに返せる石の個数、第6チャネルに隅=1、
    第7チャネルに1埋め、第8チャネルに0埋め、第9チャネルに手番情報(黒番=0埋め、白番=1埋め)
    """
    b = np.zeros((9,8,8), dtype=np.float32)
    board.piece_planes(b)
    if not board.turn:
        b = b[[1,0,2,3,4,5,6,7,8],:,:]
        b[8] = 1
    b[2] = np.where(b[0]+b[1]==1, 0, 1)
    legal_moves = list(board.legal_moves)
    if legal_moves != [64]:
        n_returns = []
        for move in legal_moves:
            board_ = copy(board)
            n_before = board_.opponent_piece_num()
            board_.move(move)
            n_after = board_.piece_num()
            n_returns.append(n_before-n_after)
        tmp = np.zeros(64)
        tmp[legal_moves] = n_returns
        tmp = tmp.reshape(8,8)
        b[3] = np.where(tmp > 0,1,0)
        b[4] = tmp
    b[5] = np.array([1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 
                     0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 
                     0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                     1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1.]).reshape(8,8)
    b[6] = 1
    return b

In [5]:
def move_rotate(move, k):
    if k == 1:
        return move_rotate270(move)
    if k == 2:
        return move_rotate180(move)
    if k == 3:
        return move_rotate90(move)
    
def move_fliplr(move):
    row = move // 8
    col = move % 8

    reversed_col = 7 - col
    reversed_move = row * 8 + reversed_col
    return reversed_move

In [6]:
class PolicyNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        n_filters = 100
        self.input_layer = nn.Sequential(
            nn.Conv2d(8,n_filters,kernel_size=5,padding=2),
            nn.ReLU()
        )
        self.hidden_layer = nn.Sequential(
            nn.Conv2d(n_filters,n_filters,kernel_size=3,padding=1),
            nn.BatchNorm2d(n_filters),
            nn.ReLU(),
            nn.Conv2d(n_filters,n_filters,kernel_size=3,padding=1),
            nn.BatchNorm2d(n_filters),
            nn.ReLU(),
            nn.Conv2d(n_filters,n_filters,kernel_size=3,padding=1),
            nn.BatchNorm2d(n_filters),
            nn.ReLU(),
            nn.Conv2d(n_filters,n_filters,kernel_size=3,padding=1),
            nn.BatchNorm2d(n_filters),
            nn.ReLU(),
            nn.Conv2d(n_filters,n_filters,kernel_size=3,padding=1),
            nn.BatchNorm2d(n_filters),
            nn.ReLU(),
            nn.Conv2d(n_filters,n_filters,kernel_size=3,padding=1),
            nn.BatchNorm2d(n_filters),
            nn.ReLU(),
            nn.Conv2d(n_filters,n_filters,kernel_size=3,padding=1),
            nn.BatchNorm2d(n_filters),
            nn.ReLU(),
            nn.Conv2d(n_filters,n_filters,kernel_size=3,padding=1),
            nn.BatchNorm2d(n_filters),
            nn.ReLU(),
            nn.Conv2d(n_filters,n_filters,kernel_size=3,padding=1),
            nn.BatchNorm2d(n_filters),
            nn.ReLU(),
            nn.Conv2d(n_filters,n_filters,kernel_size=3,padding=1),
            nn.BatchNorm2d(n_filters),
            nn.ReLU(),
            nn.Conv2d(n_filters,n_filters,kernel_size=3,padding=1),
            nn.BatchNorm2d(n_filters),
            nn.ReLU()
        )
        self.output_layer = nn.Sequential(
            nn.Conv2d(n_filters,1,kernel_size=1),
            nn.Flatten()
        )
        
    def forward(self,x):
        out = self.input_layer(x)
        out = self.hidden_layer(out)
        out = self.output_layer(out)
        return out

# 学習データ作成

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model_SL = torch.load('/kaggle/input/reversi-datasets/SL-PolicyNetwork-v3-checkpoint-5epoch-subdata99.pth').to(device)

cpu


In [8]:
def model_move(board, legal_moves, model):
    with torch.no_grad():
        p = model(torch.from_numpy(board_to_array(board)).unsqueeze(0).to(device)).cpu()
    p_legal = p[0][legal_moves].softmax(0).numpy()
    move = np.random.choice(legal_moves, p=p_legal)
    return move

In [9]:
# 1ファイル当たりの対局数
N = 50000
N_file = 7

for i in range(N_file):
    print(f'----file{i+1}----')
    S,z = [],[]
    for n in range(N):
        board = Board()
        U = np.random.randint(1,61)
        u = 1  # 手数のカウンター
        while not board.is_game_over():
            legal_moves = list(board.legal_moves)
            if 64 in legal_moves: # パスの処理
                board.move_pass()
                if u == U:
                    S.append(board_to_array2(board))
                    turn = board.turn
            elif u < U:
                move = np.random.choice(legal_moves)
                board.move(move)
            elif u == U:
                move = np.random.choice(legal_moves)
                board.move(move)
                S.append(board_to_array2(board))
                turn = board.turn
            else:
                move = model_move(board, legal_moves, model_SL)
                board.move(move)
            u += 1
        if len(S) == len(z):
            print('[WARNING] early gameover.')
            continue

        if board.turn == turn:
            z.append(1 if board.diff_num()>0 else (-1 if board.diff_num()<0 else 0))
        else:
            z.append(1 if board.diff_num()<0 else (-1 if board.diff_num()>0 else 0))
            

    S = np.array(S)
    z = np.array(z).astype(np.float32)
    np.save(f'S-{str(i+1).zfill(3)}-data-for-ValueNetwork.npy', S)
    np.save(f'z-{str(i+1).zfill(3)}-data-for-ValueNetwork.npy', z)
    print(f'S:{S.shape}, z:{z.shape}')
    print(f'pos:{np.where(z>0,1,0).sum()}, neg:{np.where(z<0,1,0).sum()}, draw:{np.where(z==0,1,0).sum()}')

----file1----
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
[WARNING] early gameover.
S:(49966, 9, 8, 8), z:(49966,)
pos:25412, neg:22775, draw:1779
----file2----
[WARNING] early gameover.